In [9]:
import pandas as pd
import numpy as np
import math
import os

# ATTENTION : under windows split on "\\", under linux split on "/"
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir("..")
print(os.getcwd())

from matplotlib import pyplot as plt
import seaborn as sns

import utils

/home/lola/Documents/Études/EPFL/MA2/Data Visualization/MigrationViz


In [11]:
# Initialization
PATH_WORLD = "data_world/"
SEX_DESTINATION_ORIGIN = PATH_WORLD + "undesa_pd_2020_ims_stock_by_sex_destination_and_origin.xlsx"

In [96]:
# Import table
sankey_2020 = pd.read_excel(SEX_DESTINATION_ORIGIN, sheet_name="Table 2", index_col=0, skiprows=9, na_values='..')

# Keep only lines of interest
sankey_2020 = sankey_2020[22:]

# Drop columns of no interest (keep only continent and sub-continent)
column_index = [1, 3] + [x for x in range(4,24)]
columns_to_drop = sankey_2020.columns[column_index]
sankey_2020.drop(axis='columns', columns=columns_to_drop, inplace=True)

# Rename columns
sankey_2020.rename(columns={"Region, development group of destination": "region",
                            "Unnamed: 3":"location_code",
                            "LATIN AMERICA AND THE CARIBBEAN": "LATIN AMERICA"}, inplace=True)

# Remove .1 and * details
sankey_2020.columns = sankey_2020.columns.str.replace(r'[*\.1]', '', regex=True)
sankey_2020["region"] = sankey_2020.region.str.replace("*","")

# Remove stupid space in front of names
column_list = list()
for col in sankey_2020.columns:
    if col.startswith(" "):
        col = " ".join(col.split())
        column_list.append(col)
    else :
        column_list.append(col)

sankey_2020.columns = column_list

30
30


In [148]:
# Northern america is both a continent and a subregion, it must be present in both continent list and sub_continent 
# Due to incomplete numbers for other sub_continents of Oceania except Australia & New Zealand, Oceania will be used both as a sub_continent and a continent
# OTHER, representing 12 millions people world-wide, needs to be taken into account every where. TODO Make sure it does not cause problems later number wise 

# 
continent = ["AFRICA", "ASIA", "EUROPE", "LATIN AMERICA AND THE CARIBBEAN", "NORTHERN AMERICA", "OCEANIA", "OTHER"]
common_columns = ["region", "location_code"]
continent_subregion = ["NORTHERN AMERICA", "OCEANIA", "OTHER"]
sub_continent = [x for x in column_list if x not in continent and x not in common_columns]

sub_continent.insert(0, common_columns[1])
sub_continent.insert(0, common_columns[0])

sub_continent.insert(19, continent_subregion[0])
sub_continent.insert(20, continent_subregion[1])

sub_continent.insert(len(sub_continent), continent_subregion[-1])

In [149]:
sub_continent

['region',
 'location_code',
 'Eastern Africa',
 'Middle Africa',
 'Northern Africa',
 'Southern Africa',
 'Western Africa',
 'Central Asia',
 'Eastern Asia',
 'South-Eastern Asia',
 'Southern Asia',
 'Western Asia',
 'Eastern Europe',
 'Northern Europe',
 'Southern Europe',
 'Western Europe',
 'Caribbean',
 'Central America',
 'South America',
 'NORTHERN AMERICA',
 'OCEANIA',
 'Australia and New Zealand',
 'Melanesia',
 'Micronesia',
 'Polynesia',
 'OTHER']

In [133]:
column_list

['region',
 'location_code',
 'AFRICA',
 'Eastern Africa',
 'Middle Africa',
 'Northern Africa',
 'Southern Africa',
 'Western Africa',
 'ASIA',
 'Central Asia',
 'Eastern Asia',
 'South-Eastern Asia',
 'Southern Asia',
 'Western Asia',
 'EUROPE',
 'Eastern Europe',
 'Northern Europe',
 'Southern Europe',
 'Western Europe',
 'LATIN AMERICA AND THE CARIBBEAN',
 'Caribbean',
 'Central America',
 'South America',
 'NORTHERN AMERICA',
 'OCEANIA',
 'Australia and New Zealand',
 'Melanesia',
 'Micronesia',
 'Polynesia',
 'OTHER']

In [111]:
sankey_2020_continent = sankey_2020.copy(deep=True)
sankey_2020_continent[common_columns + sub_continent]

,region,location_code,Eastern Africa,Middle Africa,Northern Africa,Southern Africa,Western Africa,Central Asia,Eastern Asia,South-Eastern Asia,...,Caribbean,Central America,South America,Australia and New Zealand,Melanesia,Micronesia,Polynesia,NORTHERN AMERICA,OCEANIA,OTHER
22,AFRICA,903,8244689,3300476,1539224,436677,7396499,6700,50202,51456,...,13714,2089,16721,12415,146,82,1840,53563,14483,2515243
23,Eastern Africa,910,4989040,1251025,708035,66040,17483,0,7979,1163,...,57,0,0,373,0,0,0,3935,373,421867
24,Middle Africa,911,436882,1570589,401250,7249,541785,0,1641,0,...,531,0,0,10,0,0,0,2938,10,832165
25,Northern Africa,912,1379123,144667,354273,1180,44777,4512,8993,41271,...,29,1038,778,1361,0,0,0,23989,1361,92781
26,Southern Africa,913,1433038,187553,12321,360129,54455,42,21637,7449,...,3442,647,9685,10626,121,53,23,17196,10823,665583
27,Western Africa,914,6606,146642,63345,2079,6737999,2146,9952,1573,...,9655,404,6258,45,25,29,1817,5505,1916,502847
28,ASIA,935,696185,11235,3955407,20174,37102,646620,6681690,14944523,...,501,5863,408294,99777,842,1106,0,538199,101725,4176425
29,Central Asia,5500,21,0,128,0,20,484776,103107,123,...,20,0,0,107,0,0,0,320,107,357511
30,Eastern Asia,906,0,0,1959,2160,7260,59664,5342694,2040250,...,0,2735,328942,40224,0,0,0,215678,40224,549438
31,South-Eastern Asia,920,3492,689,3756,864,397,0,896162,7088821,...,145,227,1389,43908,62,1106,0,90225,45076,641093
